# VGG16 Transfer learning - using tutorial

In [1]:
# import the necessary packages
import os

In [7]:
# set the matplotlib backend so figures can be saved in the background
import matplotlib
matplotlib.use("Agg")
# import the necessary packages
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import SGD
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
#from pyimagesearch.learningratefinder import LearningRateFinder
#from pyimagesearch.clr_callback import CyclicLR
#from pyimagesearch import config
#from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import argparse
import pickle
#import cv2
import sys
import os

#To ignore warnings
import warnings
warnings.filterwarnings('ignore')

In [3]:
train_dir = '../../data/train'
validation_dir = '../../data/dev'
test_dir = '../../data/test'

In [4]:
# define the minimum learning rate, maximum learning rate, batch size,
# step size, CLR method, and number of epochs
MIN_LR = 1e-6
MAX_LR = 1e-4
BATCH_SIZE = 32
STEP_SIZE = 8
CLR_METHOD = "triangular"
NUM_EPOCHS = 48

In [4]:
# set the path to the serialized model after training
MODEL_PATH = os.path.sep.join(["output", "natural_disaster.model"])

# define the path to the output learning rate finder plot, training
# history plot and cyclical learning rate plot
LRFIND_PLOT_PATH = os.path.sep.join(["output", "lrfind_plot.png"])
TRAINING_PLOT_PATH = os.path.sep.join(["output", "training_plot.png"])
CLR_PLOT_PATH = os.path.sep.join(["output", "clr_plot.png"])

In [ ]:
# partition the data into training and testing splits
(trainX, testX, trainY, testY) = train_test_split(data, labels,
    test_size=config.TEST_SPLIT, random_state=42)

# take the validation split from the training split
(trainX, valX, trainY, valY) = train_test_split(trainX, trainY,
    test_size=config.VAL_SPLIT, random_state=84)

In [5]:
aug = ImageDataGenerator(
    rotation_range=30,
    zoom_range=0.15,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.15,
    horizontal_flip=True,
    fill_mode="nearest")

trdata = aug
traindata = trdata.flow_from_directory(train_dir,target_size=(224,224))

tsdata = aug
testdata = tsdata.flow_from_directory(validation_dir, target_size=(224,224))

Found 12422 images belonging to 7 classes.
Found 1458 images belonging to 7 classes.


In [6]:
# load the VGG16 network, ensuring the head FC layer sets are left off
baseModel = VGG16(weights="imagenet", include_top=False,
    input_tensor=Input(shape=(224, 224, 3)))


# construct the head of the model that will be placed on top of the base model
headModel = baseModel.output
headModel = Flatten(name="flatten")(headModel)
headModel = Dense(512, activation="relu")(headModel)
headModel = Dropout(0.5)(headModel)
headModel = Dense(7, activation="softmax")(headModel)


# place the head FC model on top of the base model (this will become the actual model we will train)
model = Model(inputs=baseModel.input, outputs=headModel)


# loop over all layers in the base model and freeze them so they will *not* be updated during the first training process
for layer in baseModel.layers:
    layer.trainable = False
    
    
# compile our model (this needs to be done after our setting our layers to being non-trainable
print("[INFO] compiling model...")
opt = SGD(lr=MIN_LR, momentum=0.9)
model.compile(loss="categorical_crossentropy", optimizer=opt,
    metrics=["accuracy"])

58892288/58889256 [==============================] - 55s 1us/step
[INFO] compiling model...


In [9]:

model.fit_generator(generator= traindata, 
                          steps_per_epoch= STEP_SIZE, 
                          epochs= NUM_EPOCHS, 
                          validation_data= testdata)


Epoch 1/48
2/8 [======>.......................] - ETA: 6:02 - loss: 11.2140 - accuracy: 0.2031

KeyboardInterrupt: 